# Download dos casos de Raiva 

#### Os casos de Raiva foram baixados a partir do repositório do Sistema de Informação de Agravos de Notificação (SINAN),  que é alimentado, principalmente, pela notificação e investigação de casos de doenças e agravos que constam da lista nacional de doenças de notificação compulsória.  O downlod foi feito a partir do Package Python PySUS, criado para auxiliar no acesso e manipulação de base de dados publicadas pelo DATASUS.

#### Maiores informações em:

 - [SINAN] (https://portalsinan.saude.gov.br/sinan-net)
 - [Raiva Humana] (https://portalsinan.saude.gov.br/raiva-humana)
 - [PySUS] (https://pysus.readthedocs.io/en/latest/)


#### 1 - Importando o método SINAN do pacote PySUS

In [1]:
from pysus.ftp.databases.sinan import SINAN


#### 2 - Carregando os dados a partir do servidor do DataSUS

In [2]:
sinan = SINAN().load()

#### 3 - Lista os arquivos disponíveis para Raiva

In [3]:
arquivos = sinan.get_files(dis_code=["RAIV"])

#### 4 - Download dos arquivos disponíveis

In [4]:
import pandas as pd
import pyarrow
import pyarrow.parquet as pq

In [5]:
import os

In [6]:
parquets = sinan.download(arquivos)

100%|██████████| 14.3k/14.3k [00:00<00:00, 8.23MB/s]


In [7]:
files = []
input_folder = "/home/limafb/pysus/"
target_path = "/home/limafb/pysus/sinan_raiva.parquet"

for file_name in os.listdir(input_folder):
    if file_name.startswith('RAIV'):
        files.append(pq.read_table(os.path.join(input_folder, file_name)))

with pq.ParquetWriter(
    target_path, 
    files[0].schema, 
    version='2.6', 
    compression='gzip', 
    use_dictionary=True, 
    data_page_size=2097152, #2MB 
    write_statistics=True) as writer:
        for f in files:
              writer.write_table(f)

In [8]:
sinan_raiva = pq.read_table("sinan_raiva.parquet")

In [9]:
sinan_raiva.to_pandas()

,TP_NOT,ID_AGRAVO,DT_NOTIFIC,SEM_NOT,NU_ANO,SG_UF_NOT,ID_MUNICIP,ID_REGIONA,ID_UNIDADE,DT_SIN_PRI,...,DOENCA_TRA,DT_OBITO,DT_ENCERRA,DT_DIGITA,DT_TRANSUS,DT_TRANSDM,DT_TRANSSM,DT_TRANSRM,DT_TRANSRS,DT_TRANSSE
0,2,A829,20170103,201701,2017,52,520140,1781,2589613,20170103,...,,,20171110,20181123,,,,,,
1,2,A829,20170105,201701,2017,24,240810,1416,7408765,20170105,...,,,20170112,20170112,20170120,,,,,
2,2,A829,20170109,201702,2017,25,250090,1421,2612577,20161231,...,,,20170314,20170131,,,20170131,,,
3,2,A829,20170109,201702,2017,52,520140,1781,2589613,20170109,...,,,20171107,20181123,,,,,,
4,2,A829,20170111,201702,2017,24,240810,1416,7408765,20170111,...,,,20170112,20170112,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1674,2,A829,20151210,201549,2015,25,251650,1421,2607468,20151210,...,,,20160217,20160127,,,20160127,,,
1675,2,A829,20151217,201550,2015,50,500190,6255,2371782,20151214,...,,,20160216,20160111,,,20160111,,,
1676,2,A829,20151217,201550,2015,35,352370,1343,2745747,20151210,...,,,20160502,20151221,,,20151221,,,
1677,2,A829,20151228,201552,2015,42,420570,1554,2555050,20151228,...,,,20160108,20160108,,,20160111,,,
